# Семинар 6: функции и сортировка

### Функции

Функцией будем считать некоторый обособленный кусок кода, который можно вызвать из любой другой части кода.

In [1]:
def has_negative_number(list_numbers):
    for num in list_numbers:
        if num < 0:
            # ключевое слово return сразу говорит питону выйти из функции и вернуть значение
            return True

    return False  # вопрос: что вернет функция, если я забуду здесь return False написать?

In [16]:
from typing import List


# тайпинги все еще сохраняют динамическую типизацию,
# но позволяют улучшить читаемость кода
# чекеры вроде mypy умеют анализировать код и проверять их корректность

def has_negative_number(list_numbers: List[int]) -> bool:
    for num in list_numbers:
        if num < 0:
            return True

    return False

In [7]:
with open("numbers.txt", "r") as fin:
    while (line := fin.readline()):
        numbers = list(map(int, line.split()))
        print(f"{numbers} -> {has_negative_number(numbers)}")

[1, 2, 3, 4, 5] -> False
[5, 9] -> False
[2, -1, 5, 4, 2] -> True
[7, 8, 0, -5] -> True


Функцию можно сделать чуть более удобной для использования не только списком:

In [8]:
# все аргументы через * типизируются без List
# https://peps.python.org/pep-0484/#arbitrary-argument-lists-and-default-argument-values

def has_negative_number(*list_numbers: int) -> bool:
    for num in list_numbers:
        if num < 0:
            return True

    return False

print(has_negative_number(1, 2, 3, 4, -1))  # теперь можно не передавать список

True


Помимо `*args` есть еще и `**kwargs`:

In [11]:
from typing import Any


# на самом деле, в качестве тайпинга для values подошло бы
# какое-нибудь самописное comparable, но мы тут не будем с этим заморачиваться
# https://stackoverflow.com/questions/37669222/how-can-i-hint-that-a-type-is-comparable-with-typing


def max_value(*values: Any, **params: bool):
    return_idx = params.get("return_idx", False)  # достаем позиционный аргумент

    if not values:  # если список пустой, вернем None
        return None

    max_value = values[0]
    max_value_idx = 0
    for i, value in enumerate(values):
        if value > max_value:
            max_value = value
            max_value_idx = i

    if return_idx:
        return max_value_idx
    return max_value

In [15]:
print("max value:", max_value(6, -1, 2, 9, 1))  # сам максимум
print("max value index:", max_value(6, -1, 2, 9, 1, return_idx=True)) # индекс максимума

max value: 9
max value index: 3


### Области видимости

По умолчанию, все переменные в питоне, которые объявлены внутри функции, локальные. Вот парочка игрушечных примеров

In [31]:
def f(x):
    square_x = x ** 2

f(10)

print(square_x)  # ну конечно, вам скорее всего даже редактор подчеркнет, что так низя

NameError: name 'square_x' is not defined

In [32]:
VALUE = 10

def f():
    print(VALUE ** 2)  # но читать переменные из глобальной области можно

f()

100


In [33]:
VALUE = 10

def f():
    VALUE += 1  # но примитивы менять низя (вы зачем вообще это делать хотите???)

f()

UnboundLocalError: local variable 'CONSTANT_VALUE' referenced before assignment

In [ ]:
VALUE = 10

def f():
    global VALUE  # глобал -- это очень и очень плохо
    # иногда полезно делать global для каких-то совсем важных штук, но это редкость
    VALUE += 1  # но если очень хочется, то все-таки можно

f()

In [38]:
credentials = None

def create_user():
    global credentials  # изначально None и поэтому просто так не обратимся

    credentials = {
        "login": "destroyer_2007",
        "password": "zavtra_v_shkolu((((",
    }

print(credentials)

create_user()

print(credentials)


None
{'login': 'destroyer_2007', 'password': 'zavtra_v_shkolu(((('}


In [36]:
a = [1, 2, 3]

def f():
    a.append(4)  # но вы же не думали, что все так просто... подумайте, почему так происходит

f()

print(a)

[1, 2, 3, 4]


In [37]:
def f(a: list):  # равно так же как и поменяет список передача списка в функцию (ссылки же)
    a.append(4)

a = [1, 2, 3]

f(a)

print(a)

[1, 2, 3, 4]


### Лямбды

Анонимные однострочные функции

In [19]:
# вообще, присваивать лямбды переменным плохой тон

is_negative = lambda x: x < 0

is_negative(-1)

True

In [21]:
def square(x: int):
    return x ** 2


values = [1, 2, 3, 4, 5, 6]

print(list(map(square, values)))

[1, 4, 9, 16, 25, 36]


In [23]:
values = [1, 2, 3, 4, 5, 6]

print(list(map(lambda x: x ** 2, values)))

[1, 4, 9, 16, 25, 36]


### Сортировка

Мы уже говорили про сортировку списков, но не говорили, что ей можно задать кастомный ключ сравнения через функции

In [25]:
values = ["abcd", "aab", "bda", "0xabadbabe", "0xdeadbeef"]

sorted(values)  # все то же самое далее можно делать и с values.sort()

['0xabadbabe', '0xdeadbeef', 'aab', 'abcd', 'bda']

In [26]:
values = ["abcd", "aab", "bda", "0xabadbabe", "0xdeadbeef"]

sorted(values, key=lambda s: len(s))  # все то же самое далее можно делать и с values.sort()

['aab', 'bda', 'abcd', '0xabadbabe', '0xdeadbeef']

А как нам сравнивать по нескольким значениям сразу? Например, есть числа, хотим отсортировать их по возрастанию длин, но при равенстве -- по убыванию самих чисел

In [29]:
values = [7876510, 678, 456789, 789, 123456]

sorted(values, key=lambda num: (len(str(num)), -num))  # кортежи сравниваются поэлементно, поэтому это так и работает

[789, 678, 456789, 123456, 7876510]

# Практика

### Задача 1
Дано натуральное число $n > 1$. Выведите его наименьший делитель, отличный от 1. Решение оформите в виде функции `min_divisor(n)`. Алгоритм должен иметь сложность $O(\sqrt n)$. Указание. Если у числа $n$ нет делителя не превосходящего $\sqrt n$, то число $n$ — простое и ответом будет само число $n$.

In [ ]:
def min_divisor(n: int) -> int:
    ...

### Задача 2

Дано действительное положительное число $a$ и целоe число $n$. Вычислите $a^n$. Решение оформите в виде функции `power(a, n)`. Стандартной функцией возведения в степерь пользоваться нельзя.

In [ ]:
from typing import Union


def power(a: Union[float, int], n: int) -> Union[float, int]:
    ...

### Задача 3
Даны две точки $(x_1, y_1)$ и $(x_2, y_2)$ с целочисленными координатами. Напишите функцию `in_one_quarter`, принимающие на себя две точки и возвращающие, лежат ли две точки в одних и тех же координатных четвертях.

In [ ]:
from typing import Tuple


def is_one_quarter(one_point: Tuple[int, int], another_point: Tuple[int, int]) -> bool:
    ...

### Задача 4
Известно, что фамилии всех участников олимпиады — различны. Сохраните в массивах список всех участников и выведите его, отсортировав по фамилии в лексикографическом порядке.

При выводе указываете фамилию, имя участника и его балл.

**Ввод:**
```
Иванов Сергей 14 56
Сергеев Петр 23 74
Петров Василий 3 99
Васильев Андрей 3 56
Андреев Роман 14 75
Романов Иван 27 68
```

**Вывод:**
```
Андреев Роман 75
Васильев Андрей 56
Иванов Сергей 56
Петров Василий 99
Романов Иван 68
Сергеев Петр 74
```



In [ ]:
fin = open("input.txt", "r")

participants = fin.readlines()

# YOUR CODE HERE

fin.close()